In [1]:
from nltk.corpus import wordnet

In [3]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("text_emotion.csv")
df = df.drop(["tweet_id", "author"], axis=1)
df.head(10)

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...
5,worry,Re-pinging @ghostridah14: why didn't you go to...
6,sadness,"I should be sleep, but im not! thinking about ..."
7,worry,Hmmm. http://www.djhero.com/ is down
8,sadness,@charviray Charlene my love. I miss you
9,sadness,@kelcouch I'm sorry at least it's Friday?


In [4]:
import re
sentence = []
universal_word_list = []
maximum_length = 0
for i in df.content:
    local = ""
    flag = 0
    wordList = re.sub("[^\w]", " ",  i).split()
    for j in wordList:
        if not wordnet.synsets(j):
            continue
        elif "@" in j:
            flag = flag + 1
            local = local + "[USER] "
        else:
            flag = flag + 1
            j = j.lower()
            if j not in universal_word_list:
                universal_word_list.append(j)
            local = local + j + " "
    if maximum_length <= flag:
        maximum_length = flag
    sentence.append(local)

In [5]:
print(sentence[:10])
print("Maximum length of the sentence in the dataset", maximum_length)
content = pd.DataFrame({'sentences':sentence})
print(content.head(10))
df = df.drop(["content"], axis=1)
df['content'] = content.values
df.head(10)

['i know i was bad habit earlier i started at part ', 'n bed a headache on call ', 'funeral ceremony gloomy friday ', 'wants hang out friends soon ', 'want trade someone who has houston tickets but no one will ', 're pinging why t go prom bc t like friends ', 'i be sleep but not thinking about an old friend who i want but he s married now damn amp he wants me 2 scandalous ', 'http www is down ', 'love i miss ', 'i m sorry at least it s friday ']
Maximum length of the sentence in the dataset 31
                                           sentences
0  i know i was bad habit earlier i started at part 
1                          n bed a headache on call 
2                    funeral ceremony gloomy friday 
3                       wants hang out friends soon 
4  want trade someone who has houston tickets but...
5        re pinging why t go prom bc t like friends 
6  i be sleep but not thinking about an old frien...
7                                  http www is down 
8                       

,sentiment,content
0,empty,i know i was bad habit earlier i started at part
1,sadness,n bed a headache on call
2,sadness,funeral ceremony gloomy friday
3,enthusiasm,wants hang out friends soon
4,neutral,want trade someone who has houston tickets but...
5,worry,re pinging why t go prom bc t like friends
6,sadness,i be sleep but not thinking about an old frien...
7,worry,http www is down
8,sadness,love i miss
9,sadness,i m sorry at least it s friday


In [6]:
df = df.dropna(how='all')
for i in range(0, len(df)):
    if df.sentiment[i] == "empty":
        df = df.drop(i)
df.head(10)

,sentiment,content
1,sadness,n bed a headache on call
2,sadness,funeral ceremony gloomy friday
3,enthusiasm,wants hang out friends soon
4,neutral,want trade someone who has houston tickets but...
5,worry,re pinging why t go prom bc t like friends
6,sadness,i be sleep but not thinking about an old frien...
7,worry,http www is down
8,sadness,love i miss
9,sadness,i m sorry at least it s friday
10,neutral,cant fall asleep


In [7]:
sentiments = df.sentiment.unique()
print(universal_word_list[:10])
print(sentiments)

['i', 'know', 'was', 'bad', 'habit', 'earlier', 'started', 'at', 'part', 'n']
['sadness' 'enthusiasm' 'neutral' 'worry' 'surprise' 'love' 'fun' 'hate'
 'happiness' 'boredom' 'relief' 'anger']


In [8]:
key_to_word = {i:universal_word_list[i] for i in range(0, len(universal_word_list))}
word_to_key = {val:key for (key, val) in key_to_word.items()}

In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= 40)
tokenizer.fit_on_texts(df['content'])
sequences = tokenizer.texts_to_sequences(df['content'])
data = pad_sequences(sequences, maxlen=50)
X = data

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [10]:
import pandas as pd
import numpy as np
flag = pd.Series(list(df['sentiment']))
Y = pd.get_dummies(flag)
Y = np.array(Y)

In [11]:
total = len(X)
ratio = 0.9
flag = int(total*ratio)
X_train = X[:flag]
X_test = X[flag:]
Y_train = Y[:flag]
Y_test = Y[flag:]

In [12]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from numpy import array
from keras.models import load_model

In [13]:
model = Sequential()
model.add(LSTM(25,input_shape=(50,1),return_sequences=False))#True = many to many
model.add(Dense(30,kernel_initializer="normal",activation="relu"))
model.add(Dense(20,kernel_initializer="normal",activation="relu"))
model.add(Dense(len(sentiments),kernel_initializer="normal",activation="softmax"))
model.compile(loss="mse",optimizer ="adam",metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 25)                2700      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                780       
_________________________________________________________________
dense_2 (Dense)              (None, 20)                620       
_________________________________________________________________
dense_3 (Dense)              (None, 12)                252       
Total params: 4,352
Trainable params: 4,352
Non-trainable params: 0
_________________________________________________________________


In [17]:
X = X.reshape((len(X), 50, 1))
model.fit(X,Y,epochs=1,batch_size=32,validation_split=0.05);
model.save('train_model.h5')

Train on 37214 samples, validate on 1959 samples
Epoch 1/1
37214/37214 [==============================] - 115s 3ms/step - loss: 0.0705 - acc: 0.2582 - val_loss: 0.0723 - val_acc: 0.2027


In [ ]:
from keras.models import load_model
model = load_model('train_model.h5')
yhat = model.predict(X, verbose=0)
print(yhat)